In [1]:
import math

import numpy
import pandas

import ipywidgets as widgets
from IPython.display import display, clear_output

import gensim
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import pythainlp

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans

from data_tokenizer import load_corpus

from model.new_sdc import NewSDC
from model.sdc import SDC

### Load Data

In [2]:
file_name = 'งบกลาโหม'

corpus = load_corpus('../data/' + file_name + '.txt')

print('Total documents', len(corpus))

f = open('../data/tokenized/tokenized_' + file_name + '.txt')
tokenized_corpus = eval(f.read())
f.close()

Total documents 336


### Preprocess Corpus

#### Remove Words

In [3]:
dictionary = Dictionary(tokenized_corpus)
print('origin:', len(dictionary), 'words')

dictionary.filter_extremes(no_below=2, no_above=0.7, keep_n=len(dictionary))
print('filter frequent words:', len(dictionary), 'words')

letter_words = [id for id in range(len(dictionary)) if len(dictionary[id]) <= 1] 
dictionary.filter_tokens(bad_ids=letter_words)
print('filter letter words:', len(dictionary), 'words')

stopwords = pythainlp.corpus.stopwords.words('thai')
stopwords.extend(['นี้'])
dictionary.add_documents([stopwords])
stopwords = [dictionary.token2id[word] for word in stopwords]
dictionary.filter_tokens(bad_ids=stopwords)
print('filter stop words:', len(dictionary), 'words')

origin: 1605 words
filter frequent words: 728 words
filter letter words: 721 words
filter stop words: 480 words


In [4]:
idx_corpus = [dictionary.doc2idx(doc) for doc in tokenized_corpus]

temp_corpus = []
for doc in idx_corpus:
    temp_corpus.append([dictionary[id] for id in doc if id >= 0])
idx_corpus = temp_corpus

#### Dimension Reduction

In [5]:
average_doc_size = 0
for doc in idx_corpus:
    average_doc_size += len(doc)
average_doc_size /= len(idx_corpus)
average_doc_size = math.ceil(average_doc_size)

df = dictionary.dfs
filtered_corpus = []
for doc in idx_corpus:
    new_doc = [(word, df[dictionary.token2id[word]]) for word in doc]
    new_doc.sort(reverse=True, key=lambda x: x[1])
    new_doc = new_doc[:average_doc_size]
    filtered_corpus.append([word for word, df in new_doc])

### Clustering

In [6]:
def get_onehot(corpus, weight):
    dictionary = Dictionary(corpus)
#     dictionary.filter_extremes(no_below=2, no_above=1, keep_n=len(dictionary))

    bow_corpus = [dictionary.doc2bow(doc) for doc in corpus]
    if weight == 'normal':
        weight_corpus = bow_corpus
    elif weight == 'tfidf':
        tfidf = TfidfModel(bow_corpus, smartirs='ltc')
        weight_corpus = [tfidf[doc] for doc in bow_corpus]

    unique_words = [dictionary[id] for id in range(len(dictionary))]
    array = numpy.zeros((len(corpus), len(unique_words)), dtype=float)
    for i, doc in enumerate(weight_corpus):
        for id, score in doc:
            array[i, id] = score

        if weight == 'normal' and len(doc) != 0:
#             array[i] = numpy.divide(array[i], len(idx_corpus[i]))
            array[i] = numpy.divide(array[i], len(doc))
    
    return pandas.DataFrame(array, columns=unique_words, dtype=float)

In [7]:
def generate_result(predicted_labels, marks):
    result = pandas.DataFrame()
    result['comment'] = corpus
    result['tokenized_comment'] = idx_corpus
    result['predicted_label'] = predicted_labels
    if marks:
        result['marks'] = marks
    else:
        result['marks'] = -1
    return result

In [8]:
def eval_cluster(onehot_corpus, result):
    num_cluster = numpy.unique(result['predicted_label'])[-1] + 1

    clusters = [[] for i in range(num_cluster)]
    corpus_centroid = []
    for i, label in result['predicted_label'].iteritems():
        clusters[label].append(numpy.array(onehot_corpus.iloc[i]))
        corpus_centroid.append(numpy.array(onehot_corpus.iloc[i]))
    corpus_centroid = numpy.mean(corpus_centroid, axis=0).reshape(1, -1)   

#     print('\tIntra cluster sim\tInter cluster sim\tIntra / Inter')
    compactness = 0
    centroids = []
    for i in range(num_cluster):
        size = len(clusters[i])
        if size != 0:
            centroid = numpy.mean(clusters[i], axis=0)
            centroids.append(centroid)
            centroid = centroid.reshape(1, -1)
            similarities = cosine_similarity(centroid, clusters[i])
            compactness += numpy.sum(similarities)

#             intra = numpy.sum(similarities) / size
#             inter = cosine_similarity(centroid, corpus_centroid)[0][0]
#             print(i, end='\t')
#             print(intra, end='\t')
#             print(inter, end='\t')
#             print(intra / inter)
    return compactness, centroids

In [9]:
min_samples = 7
eps = 0.32
expand_rate = 0.05
epoch = 15

onehot_corpus = get_onehot(idx_corpus, 'normal')

In [ ]:
max_compactness = 0
for i in range(epoch):
    model = NewSDC()
    _tpredicted_labels, marks = model.predict(onehot_corpus, min_samples, eps, expand_rate)

#     model = SDC()
#     _tpredicted_labels, marks = model.predict(onehot_corpus, min_samples, eps, expand_rate)
    
#     marks = None
    
#     model = DBSCAN(metric='cosine', eps=eps, min_samples=min_samples).fit(onehot_corpus)
#     _tpredicted_labels = model.labels_ + 1

#     model = KMeans(n_clusters=14).fit(onehot_corpus)
#     _tpredicted_labels = model.labels_
    
    _tresult = generate_result(_tpredicted_labels, marks)
    compactness, _tcentroids = eval_cluster(onehot_corpus, _tresult)
    
    if compactness > max_compactness:
        max_compactness = compactness
        predicted_labels = _tpredicted_labels
        result = _tresult
        centroids = _tcentroids
        
print(max_compactness)
label_count = numpy.unique(result['predicted_label'], return_counts=True)[1]
num_cluster = len(label_count)
print(label_count)

#### Iterative New SDC

In [10]:
centroids = None 
prev_label_count = None
while True:
    model = NewSDC()
    predicted_labels, marks = model.predict(onehot_corpus, min_samples, eps, expand_rate, seeds=centroids)
    
    result = generate_result(predicted_labels, marks)
    compactness, centroids = eval_cluster(onehot_corpus, result)
    
    label_count = numpy.unique(result['predicted_label'], return_counts=True)[1]
    if numpy.array_equal(label_count, prev_label_count):
        break
    prev_label_count = label_count
    centroids = centroids[1:]
    
    print(compactness)
    print(label_count)
num_cluster = len(label_count)

147.99920346689785
[83  7 18  8 41 28 41 13 17 19 11 19 11  8 12]
144.11942422399773
[81  8 52 22 44 34 33  8 17 11 15 11]
141.02258618543033
[85  8 72 19 47 34 22 16 12 12  9]
139.94507888285364
[88  8 82 16 48 34 14 15 12 11  8]
139.03378973656007
[92  8 80 16 48 34 13 15 12 11  7]


#### Grouping

In [11]:
sims = cosine_similarity(centroids)
new_labels = [i for i in range(num_cluster)]
print(new_labels)
for i, row in reversed(list(enumerate(sims))):
    for j, value in reversed(list(enumerate(row[:i + 1]))):
        if i != j and value >= eps - eps / 20:
            print(i, j, value)
            base = min(new_labels[i], new_labels[j])
            new_labels[j] = base
            new_labels = [base if label == new_labels[i] else label for label in new_labels]
print(new_labels)

grouped_labels = numpy.zeros(len(corpus))
for i, label in enumerate(predicted_labels):
    grouped_labels[i] = new_labels[label]
new_result = generate_result(grouped_labels, None)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
6 2 0.3308406224632874
[0, 1, 2, 3, 4, 5, 2, 7, 8, 9, 10]


### Result

In [12]:
class Widget:
    def __init__(self, result, column_name):
        self.result = result
        self.column_name = column_name
        
        label_count = numpy.unique(result['predicted_label'])
        self.widget = widgets.ToggleButtons(
            options=[int(num) for num in label_count],
            disabled=False,
            button_style='',
        )
        
        self.widget.observe(self.on_click, names='index')
        self.on_click({'new' : 0})
        
    def on_click(self, change):
        clear_output()
        display(self.widget)
        new = self.widget.options[change['new']]
        for index, value in self.result[self.result['predicted_label'] == new].iterrows():
            if value['marks'] == 0:
                print("@", end="")
            elif value['marks'] == 1:
                print("*", end="")
            print(index, value[self.column_name])

In [13]:
# result.to_csv('../data/results/filtered/' + file_name + '.csv', index=False)

# result = pandas.read_csv('../data/results/filtered/' + file_name + '.csv')

print(eval_cluster(onehot_corpus, result)[0])
print(numpy.unique(result['predicted_label'], return_counts=True)[1])

139.03378973656007
[92  8 80 16 48 34 13 15 12 11  7]


In [14]:
w1 = Widget(new_result, 'comment')

ToggleButtons(options=(0, 1, 2, 3, 4, 5, 7, 8, 9, 10), value=0)

2 กำลังจะเสิร์ชดูสถิติแต่ละปีเลยค่ะ🤩🤩
4 ว่าด้วยเหตุด้วยผลแบบนี้ ถูกและสง่างามที่สุด
6 ทุกกระทรวง ขึ้น หมด.
13 ชื่นชมในการนำเสนอข้อมูลอย่างตรงไปตรงมาค่ะ และอยากทราบแนวทางในการหารายได้ให้ประเทศของทางพรรค เนื่องจากทุกๆ พรรคนำเสนอแต่นโยบายที่จะให้กับประชาชน โดยเราไม่รู้เลยว่าทางพรรคจะเอารายได้จากส่วนไหนมาดำเนินการตามนโยบายนั้นๆ ซึ่งหากทางพรรคให้ข้อมูลได้ ก็จะเป็นประโยชน์อย่างยิ่งในการตัดสินใจค่ะ
14 ทำไมรู้สึกว่ามันฟังดูมีเหตุผล
22 จริงๆ สัดส่วนงบกลาโหมเราเมื่อเทียบกับ GDP ถือว่าเหมาะสมไหมครับ ในมุมมองของคุณกรณ์ เมื่อเทียบกับ อาวุธที่เรามี ส่วนมากเป็นของตกรุ่นมาจากยุคสงครามเย็น ระดับขีดความสามารถในการป้องกันประเทศเมื่อเทียบกับชาติ อาเซียน แล้วเราเป็นยังไง อันนี้ควรตั้งคำถาม และ การพิจารณาและพิจารณ์ ครับ
25 ขออนุญาติแชร์นะคะ
26 มีอีกชาร์ตที่เป็นอันดับของอาเซียนลองหาดูใน Google ไม่พูดมากเจ็บคอ
28 ชอบการนำเสนอ Fact มากกว่า Feel แบบนี้ครับ ❤️
29 อดีต รมว.คลังออกมาเอง เอ้า!ใครที่ออกมาฟาดงวงฟาดงาแสดงตัวหน่อย
34 ถ้าตีเป็นตัวเลขกลมๆแบบคร่าวๆไทย 2แสน2หมื่นล้านบาทพม่า ประมาณ 8-9หมื่นล้านบาทลาว ประมาณ

In [15]:
w2 = Widget(result, 'comment')

ToggleButtons(options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10), value=0)

2 กำลังจะเสิร์ชดูสถิติแต่ละปีเลยค่ะ🤩🤩
4 ว่าด้วยเหตุด้วยผลแบบนี้ ถูกและสง่างามที่สุด
6 ทุกกระทรวง ขึ้น หมด.
13 ชื่นชมในการนำเสนอข้อมูลอย่างตรงไปตรงมาค่ะ และอยากทราบแนวทางในการหารายได้ให้ประเทศของทางพรรค เนื่องจากทุกๆ พรรคนำเสนอแต่นโยบายที่จะให้กับประชาชน โดยเราไม่รู้เลยว่าทางพรรคจะเอารายได้จากส่วนไหนมาดำเนินการตามนโยบายนั้นๆ ซึ่งหากทางพรรคให้ข้อมูลได้ ก็จะเป็นประโยชน์อย่างยิ่งในการตัดสินใจค่ะ
14 ทำไมรู้สึกว่ามันฟังดูมีเหตุผล
22 จริงๆ สัดส่วนงบกลาโหมเราเมื่อเทียบกับ GDP ถือว่าเหมาะสมไหมครับ ในมุมมองของคุณกรณ์ เมื่อเทียบกับ อาวุธที่เรามี ส่วนมากเป็นของตกรุ่นมาจากยุคสงครามเย็น ระดับขีดความสามารถในการป้องกันประเทศเมื่อเทียบกับชาติ อาเซียน แล้วเราเป็นยังไง อันนี้ควรตั้งคำถาม และ การพิจารณาและพิจารณ์ ครับ
25 ขออนุญาติแชร์นะคะ
26 มีอีกชาร์ตที่เป็นอันดับของอาเซียนลองหาดูใน Google ไม่พูดมากเจ็บคอ
28 ชอบการนำเสนอ Fact มากกว่า Feel แบบนี้ครับ ❤️
29 อดีต รมว.คลังออกมาเอง เอ้า!ใครที่ออกมาฟาดงวงฟาดงาแสดงตัวหน่อย
34 ถ้าตีเป็นตัวเลขกลมๆแบบคร่าวๆไทย 2แสน2หมื่นล้านบาทพม่า ประมาณ 8-9หมื่นล้านบาทลาว ประมาณ

In [16]:
w3 = Widget(result, 'tokenized_comment')

ToggleButtons(index=9, options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10), value=9)

*36 ['ยุค', 'นายก', 'ยุค', 'ดี', 'รัฐบาล']
*42 ['เมือง', 'น้ำ', 'ดี', 'หา', 'เมือง', 'ประชาชน', 'ขับไล่', 'เมือง', 'ต่ำ']
*43 ['เมือง', 'ดี', 'หาเสียง', 'หลักการ', 'ไม้']
*74 ['เมือง', 'ยุค', 'ตอน', 'หาเสียง', 'พิจารณา', 'เหมาะสม', 'ดี', 'เลือก', 'อนาคต', 'ล่ะ']
*148 ['ดี', 'เพจ', 'รู้']
*155 ['เงิน', 'ดี', 'เหรอ', 'ตอน', 'กอง', 'ทัพ', 'ใช้จ่าย', 'รถ', 'ถัง', 'เมืองไทย', 'คุณภาพ', 'ดี', 'จีน', 'ทำ', 'ซื้อ']
*156 ['ดี']
*162 ['แบบอย่าง', 'ดี', 'เสนอ', 'รู้', 'ประชาธิปไตย', 'ดี', 'เหมือน', 'ด่า', 'ฟัง']
*240 ['ผม', 'ดี', 'อธิบาย', 'ข้อมูล', 'สถิติ', 'ดี', 'อธิบาย', 'ชวน', 'ประชาชน', 'กิน', 'ดี', 'อยู่ดี', 'คุณภาพ', 'เหมือน', 'ชวน', 'พรรค', 'อนุญาติแชร์']
*266 ['รายละเอียด', 'ใช้จ่าย', 'ดี', 'ตัว', 'ลด']
*281 ['เลือกตั้ง', 'วิถี', 'ระบบ', 'ประชาธิปไตย', 'เลือก', 'ขัดแย้ง', 'เลือกตั้ง', 'ดี', 'ประชาธิปไตย', 'ทะเลาะ', 'เผด็จการ', 'คืน', 'สร้าง', 'ดี', 'ประชาธิปไตย', 'วุ่นวาย', 'มัง']


In [19]:
seed = 0
compare = 0

a = numpy.array(onehot_corpus.iloc[seed]).reshape(1, -1)
b = numpy.array(onehot_corpus.iloc[compare]).reshape(1, -1)
print(cosine_similarity(a,b))

print(idx_corpus[seed])
print(corpus[seed])
print(idx_corpus[compare])
print(corpus[compare])

[[1.]]
['ประกาศ', 'หยุด', 'เรียน', 'ระดับ', 'ชั้น', 'บริหาร', 'โรง', 'เรียน', 'ประกาศ', 'ชม']
รร.รุ่งอรุณประกาศหยุดเรียนทุกระดับชั้น ชื่นชมผู้บริหารโรงเรียนในการตัดสิ้นใจค่ะ ประกาศเมื่อราว 1 ชม.ที่ผ่านมาค่ะ
['ประกาศ', 'หยุด', 'เรียน', 'ระดับ', 'ชั้น', 'บริหาร', 'โรง', 'เรียน', 'ประกาศ', 'ชม']
รร.รุ่งอรุณประกาศหยุดเรียนทุกระดับชั้น ชื่นชมผู้บริหารโรงเรียนในการตัดสิ้นใจค่ะ ประกาศเมื่อราว 1 ชม.ที่ผ่านมาค่ะ
